In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

In [2]:
import pandas as pd
import numpy as np
from src.load_data import load_data
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
from src.load_data import load_csv, load_raw_csv

# Get price by product

In [7]:
promos = load_csv("promos")

===================  LOAD DATASET promos ===================
Reading CSV in data/raw/02_TablaPromos.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN PROMOS  ============
Drop rows of promos without end date:
   Rows dropped: 11
   The most recent of the promos dropped was started in 2018-04-01 00:00:00
Created new variable ahorro (preciotarifa - preciopromo)
Dataset promos loaded. Shape: (6399, 11)



In [59]:
prods = []
tarifas = []

for prod in promos.producto.unique():
    tarifa = promos.loc[promos.producto == prod, "preciotarifa"].tail(1).values[0]
    tarifas.append(round(tarifa,2))
    prods.append(prod)
    
precios = pd.DataFrame({"producto":prods, "precio":tarifas}).sort_values("producto").reset_index(drop=True)

precios.head()

,producto,precio
0,1,3.78
1,10,3.39
2,11,3.20
3,12,4.35
4,13,3.93


In [60]:
precios.to_csv("data/clean/precios.csv", index=False)

# Stock real

In [61]:
stock = load_raw_csv("stock_real")

stock.head()

Reading CSV in data/raw/03_TablaStockReal.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 314.


,fecha,producto,udsstock
6597,2019-06-05,49,222
6598,2019-06-05,91,104
6596,2019-06-05,3,27112
6594,2019-06-05,7,208
6593,2019-06-05,63,578


# Conform Business Case Data

In [153]:
preds = pd.read_csv("predict.csv", parse_dates=["fecha"], dtype={"producto":str})
preds["type"] = preds.modelo.apply(lambda x: x[:3])

preds

,y_pred,y_real,type,fecha,producto,modelo
0,17460.670,18275.0,ALL,2019-06-05,1,ALL
1,3553.240,2920.0,ALL,2019-06-11,1,ALL
2,6558.955,6731.0,ALL,2019-06-12,1,ALL
3,7922.845,7241.0,ALL,2019-06-13,1,ALL
4,4757.160,3850.0,ALL,2019-06-14,1,ALL
...,...,...,...,...,...,...
32603,245.255,3.0,PR_,2020-03-17,98,PR_98
32604,317.745,90.0,PR_,2020-03-18,98,PR_98
32605,509.055,714.0,PR_,2020-03-19,98,PR_98
32606,578.395,786.0,PR_,2020-03-20,98,PR_98


In [154]:
from src.load_data import load_csv
df = load_csv("stock")
df = df.groupby(["producto","fecha"])['udsstock'].agg(['min',"mean", 'max']).reset_index()
df.columns = ["producto","fecha","stockmin","stockmean","stockmax"]
preds = preds.merge(df, how='left',on=["fecha","producto"]).set_index("fecha")
ventas = load_csv("ventas")
ventas['udsventa'] = ventas['udsventa'].fillna(0)

preds = preds.merge(ventas, how = 'left',on=["fecha","producto"])
preds = preds.merge(stock, how='left',on=["fecha","producto"])
preds = preds.merge(precios, how='left',on="producto")
preds['udsventa'] = preds["udsventa"].fillna(0)
preds

===================  LOAD DATASET stock ====================
Reading CSV in data/raw/03_TablaStock.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 628.
============  CLEAN STOCK  =============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). Rows dropped: 1764
[WARNING] Number of dates missings:                                83
[WARNING] Number of dates missings (droping sundays and festivos): 63
[WARNING] Stock data with different units for same product & data. Rows: 12679
Dataset stock loaded. Shape: (25538, 3)

===================  LOAD DATASET ventas ===================
Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN VENTAS  ============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). 

,fecha,y_pred,y_real,type,producto,modelo,stockmin,stockmean,stockmax,udsventa,udsstock,precio
0,2019-06-05,17460.670,18275.0,ALL,1,ALL,13919.0,18275.5,22632.0,2833.0,22632,3.78
1,2019-06-11,3553.240,2920.0,ALL,1,ALL,2224.0,2920.0,3616.0,1416.0,3616,3.78
2,2019-06-12,6558.955,6731.0,ALL,1,ALL,5127.0,6731.5,8336.0,1889.0,8336,3.78
3,2019-06-13,7922.845,7241.0,ALL,1,ALL,5515.0,7241.5,8968.0,5667.0,8968,3.78
4,2019-06-14,4757.160,3850.0,ALL,1,ALL,2932.0,3850.0,4768.0,3778.0,4768,3.78
...,...,...,...,...,...,...,...,...,...,...,...,...
32603,2020-03-17,245.255,3.0,PR_,98,PR_98,2.0,3.0,4.0,637.0,4,3.31
32604,2020-03-18,317.745,90.0,PR_,98,PR_98,69.0,90.5,112.0,565.0,112,3.31
32605,2020-03-19,509.055,714.0,PR_,98,PR_98,544.0,714.0,884.0,324.0,884,3.31
32606,2020-03-20,578.395,786.0,PR_,98,PR_98,599.0,786.5,974.0,391.0,974,3.31


In [156]:
preds.to_csv("businesscase.csv", index=False)

In [172]:
def get_costes(df, stock):

    df["balance"] = df[stock] - df["udsventa"]
    df["coste"] = np.abs(df["precio"] * df["balance"])
    df["coste_sobrestock"] = (df["balance"]>0) * df["coste"] * 0.15
    df["coste_rotura"] = (df["balance"]<0) * df["coste"] * 0.22
    df["revenue"] = df["udsventa"] * df["precio"]
    print("Numero de roturas de stock: {}".format((df["balance"]<0).sum()))
    print("Volumen de negocio en el periodo y productos evaluados: {}".format(df.revenue.sum()))
    print("Coste por sobreestock total: {}".format(df["coste_sobrestock"].sum()))
    print("Coste por rotura de stock total: {}".format(df["coste_rotura"].sum()))
    print("Coste por sobreestock total: {}".format(df["coste_sobrestock"].sum()))
    print("Coste por rotura de stock media diaria: {}".format(df.groupby("fecha")["coste_rotura"].sum().mean()))
    print("Coste por sobrestock media diaria: {}".format(df.groupby("fecha")["coste_sobrestock"].sum().mean()))
    
    return df

## Costes del método actual

In [173]:
coste_real = preds.groupby(["fecha","producto"])["udsstock","udsventa", "precio"].min().reset_index()
coste_real

/home/plombao/miniconda3/envs/ds_all/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,fecha,producto,udsstock,udsventa,precio
0,2019-06-05,1,22632,2833.0,3.78
1,2019-06-05,10,2808,1874.0,3.39
2,2019-06-05,11,3104,2755.0,3.20
3,2019-06-05,12,1584,1161.0,4.35
4,2019-06-05,13,3088,1603.0,3.93
...,...,...,...,...,...
11141,2020-03-21,91,4,0.0,4.29
11142,2020-03-21,94,706,8.0,3.95
11143,2020-03-21,96,614,36.0,5.42
11144,2020-03-21,97,676,0.0,3.83


In [174]:
get_costes(coste_real, "udsstock")

Numero de roturas de stock: 3008
Volumen de negocio en el periodo y productos evaluados: 46105702.12
Coste por sobreestock total: 6743339.994
Coste por rotura de stock total: 2473757.77
Coste por sobreestock total: 6743339.994
Coste por rotura de stock media diaria: 13819.875810055863
Coste por sobrestock media diaria: 37672.29046927375


,fecha,producto,udsstock,udsventa,precio,balance,coste,coste_sobrestock,coste_rotura,revenue
0,2019-06-05,1,22632,2833.0,3.78,19799.0,74840.22,11226.0330,0.0,10708.74
1,2019-06-05,10,2808,1874.0,3.39,934.0,3166.26,474.9390,0.0,6352.86
2,2019-06-05,11,3104,2755.0,3.20,349.0,1116.80,167.5200,0.0,8816.00
3,2019-06-05,12,1584,1161.0,4.35,423.0,1840.05,276.0075,0.0,5050.35
4,2019-06-05,13,3088,1603.0,3.93,1485.0,5836.05,875.4075,0.0,6299.79
...,...,...,...,...,...,...,...,...,...,...
11141,2020-03-21,91,4,0.0,4.29,4.0,17.16,2.5740,0.0,0.00
11142,2020-03-21,94,706,8.0,3.95,698.0,2757.10,413.5650,0.0,31.60
11143,2020-03-21,96,614,36.0,5.42,578.0,3132.76,469.9140,0.0,195.12
11144,2020-03-21,97,676,0.0,3.83,676.0,2589.08,388.3620,0.0,0.00


# Coste modelo unitario

In [175]:
coste_unitario = preds.loc[preds.type == "PR_", ["fecha", "producto", "y_pred", "udsventa","precio"]]

coste_unitario

,fecha,producto,y_pred,udsventa,precio
21885,2019-06-05,1,15025.280,2833.0,3.78
21886,2019-06-11,1,3964.645,1416.0,3.78
21887,2019-06-12,1,6381.135,1889.0,3.78
21888,2019-06-13,1,8053.565,5667.0,3.78
21889,2019-06-14,1,4125.100,3778.0,3.78
...,...,...,...,...,...
32603,2020-03-17,98,245.255,637.0,3.31
32604,2020-03-18,98,317.745,565.0,3.31
32605,2020-03-19,98,509.055,324.0,3.31
32606,2020-03-20,98,578.395,391.0,3.31


In [176]:
get_costes(coste_unitario, "y_pred")

Numero de roturas de stock: 3226
Volumen de negocio en el periodo y productos evaluados: 44618469.269999996
Coste por sobreestock total: 4372185.3870075
Coste por rotura de stock total: 2385698.240739
Coste por sobreestock total: 4372185.3870075
Coste por rotura de stock media diaria: 13327.923132620115
Coste por sobrestock media diaria: 24425.61668719274


,fecha,producto,y_pred,udsventa,precio,balance,coste,coste_sobrestock,coste_rotura,revenue
21885,2019-06-05,1,15025.280,2833.0,3.78,12192.280,46086.81840,6913.022760,0.000000,10708.74
21886,2019-06-11,1,3964.645,1416.0,3.78,2548.645,9633.87810,1445.081715,0.000000,5352.48
21887,2019-06-12,1,6381.135,1889.0,3.78,4492.135,16980.27030,2547.040545,0.000000,7140.42
21888,2019-06-13,1,8053.565,5667.0,3.78,2386.565,9021.21570,1353.182355,0.000000,21421.26
21889,2019-06-14,1,4125.100,3778.0,3.78,347.100,1312.03800,196.805700,0.000000,14280.84
...,...,...,...,...,...,...,...,...,...,...
32603,2020-03-17,98,245.255,637.0,3.31,-391.745,1296.67595,0.000000,285.268709,2108.47
32604,2020-03-18,98,317.745,565.0,3.31,-247.255,818.41405,0.000000,180.051091,1870.15
32605,2020-03-19,98,509.055,324.0,3.31,185.055,612.53205,91.879807,0.000000,1072.44
32606,2020-03-20,98,578.395,391.0,3.31,187.395,620.27745,93.041617,0.000000,1294.21


# Coste modelo CLUSTERING

In [177]:
coste_cluster = preds.loc[preds.type == "CL_", ["fecha", "producto", "y_pred", "udsventa","precio"]]

coste_cluster

,fecha,producto,y_pred,udsventa,precio
10723,2019-06-05,23,1477.170,2201.0,3.70
10724,2019-06-11,23,1779.110,953.0,3.70
10725,2019-06-12,23,2450.420,1975.0,3.70
10726,2019-06-13,23,2961.595,1248.0,3.70
10727,2019-06-14,23,3386.540,1832.0,3.70
...,...,...,...,...,...
21880,2020-03-17,5,5975.320,10782.0,2.35
21881,2020-03-18,5,5858.900,8804.0,2.35
21882,2020-03-19,5,5112.930,6464.0,2.35
21883,2020-03-20,5,4418.325,8708.0,2.35


In [178]:
get_costes(coste_cluster, "y_pred")

Numero de roturas de stock: 3391
Volumen de negocio en el periodo y productos evaluados: 45601970.65
Coste por sobreestock total: 4419207.9345525
Coste por rotura de stock total: 2544412.263547
Coste por sobreestock total: 4419207.9345525
Coste por rotura de stock media diaria: 14214.593651100557
Coste por sobrestock media diaria: 24688.31248353352


,fecha,producto,y_pred,udsventa,precio,balance,coste,coste_sobrestock,coste_rotura,revenue
10723,2019-06-05,23,1477.170,2201.0,3.70,-723.830,2678.17100,0.000000,589.197620,8143.70
10724,2019-06-11,23,1779.110,953.0,3.70,826.110,3056.60700,458.491050,0.000000,3526.10
10725,2019-06-12,23,2450.420,1975.0,3.70,475.420,1759.05400,263.858100,0.000000,7307.50
10726,2019-06-13,23,2961.595,1248.0,3.70,1713.595,6340.30150,951.045225,0.000000,4617.60
10727,2019-06-14,23,3386.540,1832.0,3.70,1554.540,5751.79800,862.769700,0.000000,6778.40
...,...,...,...,...,...,...,...,...,...,...
21880,2020-03-17,5,5975.320,10782.0,2.35,-4806.680,11295.69800,0.000000,2485.053560,25337.70
21881,2020-03-18,5,5858.900,8804.0,2.35,-2945.100,6920.98500,0.000000,1522.616700,20689.40
21882,2020-03-19,5,5112.930,6464.0,2.35,-1351.070,3175.01450,0.000000,698.503190,15190.40
21883,2020-03-20,5,4418.325,8708.0,2.35,-4289.675,10080.73625,0.000000,2217.761975,20463.80


# Coste ALL

In [179]:
coste_all = preds.loc[preds.type == "ALL", ["fecha", "producto", "y_pred", "udsventa","precio"]]

coste_all

,fecha,producto,y_pred,udsventa,precio
0,2019-06-05,1,17460.670,2833.0,3.78
1,2019-06-11,1,3553.240,1416.0,3.78
2,2019-06-12,1,6558.955,1889.0,3.78
3,2019-06-13,1,7922.845,5667.0,3.78
4,2019-06-14,1,4757.160,3778.0,3.78
...,...,...,...,...,...
10718,2020-03-17,98,246.060,637.0,3.31
10719,2020-03-18,98,254.360,565.0,3.31
10720,2020-03-19,98,744.670,324.0,3.31
10721,2020-03-20,98,705.500,391.0,3.31


In [180]:
get_costes(coste_all, "y_pred")

Numero de roturas de stock: 3284
Volumen de negocio en el periodo y productos evaluados: 44618469.269999996
Coste por sobreestock total: 4479663.4013625
Coste por rotura de stock total: 2505638.958702
Coste por sobreestock total: 4479663.4013625
Coste por rotura de stock media diaria: 13997.983009508382
Coste por sobrestock media diaria: 25026.052521578215


,fecha,producto,y_pred,udsventa,precio,balance,coste,coste_sobrestock,coste_rotura,revenue
0,2019-06-05,1,17460.670,2833.0,3.78,14627.670,55292.5926,8293.888890,0.000000,10708.74
1,2019-06-11,1,3553.240,1416.0,3.78,2137.240,8078.7672,1211.815080,0.000000,5352.48
2,2019-06-12,1,6558.955,1889.0,3.78,4669.955,17652.4299,2647.864485,0.000000,7140.42
3,2019-06-13,1,7922.845,5667.0,3.78,2255.845,8527.0941,1279.064115,0.000000,21421.26
4,2019-06-14,1,4757.160,3778.0,3.78,979.160,3701.2248,555.183720,0.000000,14280.84
...,...,...,...,...,...,...,...,...,...,...
10718,2020-03-17,98,246.060,637.0,3.31,-390.940,1294.0114,0.000000,284.682508,2108.47
10719,2020-03-18,98,254.360,565.0,3.31,-310.640,1028.2184,0.000000,226.208048,1870.15
10720,2020-03-19,98,744.670,324.0,3.31,420.670,1392.4177,208.862655,0.000000,1072.44
10721,2020-03-20,98,705.500,391.0,3.31,314.500,1040.9950,156.149250,0.000000,1294.21
